In [34]:
import pandas as pd 
import numpy as np
from IPython.core.display import HTML
from string import Template
import json
import os
import random
import math

In [35]:
HTML('''
<script src="../../sigma.js/src/sigma.core.js"></script>
<script src="../../sigma.js/src/conrad.js"></script>
<script src="../../sigma.js/src/utils/sigma.utils.js"></script>
<script src="../../sigma.js/src/utils/sigma.polyfills.js"></script>
<script src="../../sigma.js/src/sigma.settings.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.dispatcher.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.configurable.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.graph.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.camera.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.quad.js"></script>
<script src="../../sigma.js/src/classes/sigma.classes.edgequad.js"></script>
<script src="../../sigma.js/src/captors/sigma.captors.mouse.js"></script>
<script src="../../sigma.js/src/captors/sigma.captors.touch.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.canvas.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.webgl.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.svg.js"></script>
<script src="../../sigma.js/src/renderers/sigma.renderers.def.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.nodes.def.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.nodes.fast.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.edges.def.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.edges.fast.js"></script>
<script src="../../sigma.js/src/renderers/webgl/sigma.webgl.edges.arrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.labels.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.hovers.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.nodes.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.curve.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.arrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edges.curvedArrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.def.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.curve.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.arrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.edgehovers.curvedArrow.js"></script>
<script src="../../sigma.js/src/renderers/canvas/sigma.canvas.extremities.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.utils.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.nodes.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.edges.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.edges.curve.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.labels.def.js"></script>
<script src="../../sigma.js/src/renderers/svg/sigma.svg.hovers.def.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.rescale.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.copy.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.animation.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindDOMEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.drawHovers.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.rescale.js"></script>
<script src="../../sigma.js/src/middlewares/sigma.middlewares.copy.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.animation.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.bindDOMEvents.js"></script>
<script src="../../sigma.js/src/misc/sigma.misc.drawHovers.js"></script>
<script src="../../sigma.js/plugins/sigma.layout.forceAtlas2/worker.js"></script>
<script src="../../sigma.js/plugins/sigma.layout.forceAtlas2/supervisor.js"></script>
<script src="../../sigma.js/plugins/sigma.plugins.neighborhoods/sigma.plugins.neighborhoods.js"></script>
''')

In [36]:
all_cases = pd.read_csv('/Users/starice/OwnFiles/cityu/RA/case_study/data/total_extracted_result/all_cases.csv', encoding="utf-8")
# all_cases.head()

In [37]:
# The lawyer inforamtion need to be corrected
all_cases = all_cases[~(all_cases['defendant']==all_cases['lawyer'])]

<h1>Building Knowledge Graph</h1>
<p>Entities: case, plaintiff, defendant, lawyer, court, judge</p>
<p>Relationships(undirected):<br> 
    case-plaintiff: sue, <br>
    case-defendant: be sued, <br>
    plaintiff-lawyer: cooperate, <br>
    case-court: locate, <br>
    case-judge: judge, <br>
    case-lawyer: legally participate
</p>


<h2>1. How to use jupyter notebook to display html content?</h2>

In [38]:
HTML('''
<h1>Hello! HTML in Jupyter Notebook.</h1>
''')

In [39]:
HTML('''
<style>
.teststyle{
  color: blue;
  font: 16px script;
}
</style>
<h1 class="teststyle">Hello! HTML in Jupyter Notebook.</h1>
''')

<h2>2. Build sigma js example here.</h2>
<!-- <p><font color="red">Have some trouble in it.</font></p> -->

In [40]:
css_text = '''
  #graph-container {
      top: 0;
      bottom: 0;
      left: 0;
      right: 0;
      position: absolute;
    }
'''

js_text_template = Template('''
  sigma.parsers.json('$data_path.json', {
    container: '$container',
    settings: {
      defaultNodeColor: '#ec5148'
    }
  });
''')

js_text_template2 = Template('''
s = new sigma({ 
        graph: $data,
        container: '$container',
        settings: {
            defaultNodeColor: '#ec5148'
        }
});
''')

js_text_template3 = '''
var i,
    s,
    N = 100,
    E = 500,
    g = {
      nodes: [],
      edges: []
    };

for (i = 0; i < N; i++)
  g.nodes.push({
    id: 'n' + i,
    label: 'Node ' + i,
    x: Math.random(),
    y: Math.random(),
    size: Math.random(),
    color: '#666'
  });

for (i = 0; i < E; i++)
  g.edges.push({
    id: 'e' + i,
    source: 'n' + (Math.random() * N | 0),
    target: 'n' + (Math.random() * N | 0),
    size: Math.random(),
    color: '#ccc'
  });

s = new sigma({
  graph: g,
  container: 'graph-container'
});
'''

In [41]:
# symbol $ here enables variable names in string to be replaced by variable data
html_template = Template('''
<style>
  #graph-container {
      height: 800px;
  };
</style>
<div id="graph-container"></div>
<script>$js_text</script>
''') 

In [42]:
# js_text = js_text_template2.substitute({
#     "data": json.dumps(example_data), 
#     "container": "sigmajs_example"})
# # js_text

In [43]:
# html_text = html_template.substitute({
#     "css_text": css_text, 
# #     "div_id": "sigmajs_example", 
#     "js_text": js_text_template3
# })
# HTML(html_text)

<h2>3. Build graph data in json format for sigma to read</h2>

In [44]:
node_types = ['case', 'plaintiff', 'defendant', 'lawyer', 'court', 'judge']
edge_types = ["case-plaintiff", "case-defendant", "plaintiff-lawyer", "case-court", "case-judge", "case-lawyer"]

In [45]:
# randomly choose data within 5 plaintiffs
plaintiffs = list(all_cases['plaintiff'].drop_duplicates())
selected_plaintiffs = random.choices(plaintiffs, k=2)
print(selected_plaintiffs)
selected_cases = all_cases[all_cases['plaintiff'].isin(selected_plaintiffs)]

['张新艳', '孙亚']


In [46]:
selected_cases.rename(columns={
    "case_id": "case", 
    "court_name": "court"}, 
                      inplace=True
                     )
selected_cases["case-plaintiff"] = selected_cases[['case', 'plaintiff']].apply(tuple, axis=1)
selected_cases["case-defendant"] = selected_cases[['case', 'defendant']].apply(tuple, axis=1)
selected_cases["plaintiff-lawyer"] = selected_cases[['plaintiff', 'lawyer']].apply(tuple, axis=1)
selected_cases["case-court"] = selected_cases[['case', 'court']].apply(tuple, axis=1)
selected_cases["case-judge"] = selected_cases[['case', 'judge']].apply(tuple, axis=1)
selected_cases["case-lawyer"] = selected_cases[['case', 'lawyer']].apply(tuple, axis=1)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-46-cae598d79b4e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_cases["case-plaintiff"] = selected_cases[['case', 'plaintiff']].apply(tuple, axis=1)
<ipython-input-46-cae598d79b4e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [47]:
selected_cases

,Unnamed: 0,case,judgement_date,is_success,court,procedure,judge,legalfee,objectmoney,province,...,day,plaintiff,defendant,lawyer,case-plaintiff,case-defendant,plaintiff-lawyer,case-court,case-judge,case-lawyer
140,143,57a9e792c2265c25897332c9,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,王钲舜,王德军,"(57a9e792c2265c25897332c9, 张新艳)","(57a9e792c2265c25897332c9, 王钲舜)","(张新艳, 王德军)","(57a9e792c2265c25897332c9, 江苏省南京市中级人民法院)","(57a9e792c2265c25897332c9, 贡永红)","(57a9e792c2265c25897332c9, 王德军)"
141,144,57a9e792c2265c25897332c9,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,王钲舜,段然,"(57a9e792c2265c25897332c9, 张新艳)","(57a9e792c2265c25897332c9, 王钲舜)","(张新艳, 段然)","(57a9e792c2265c25897332c9, 江苏省南京市中级人民法院)","(57a9e792c2265c25897332c9, 贡永红)","(57a9e792c2265c25897332c9, 段然)"
142,145,57a9e792c2265c25897332c9,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,南京鸿翥生物科技有限公司,王德军,"(57a9e792c2265c25897332c9, 张新艳)","(57a9e792c2265c25897332c9, 南京鸿翥生物科技有限公司)","(张新艳, 王德军)","(57a9e792c2265c25897332c9, 江苏省南京市中级人民法院)","(57a9e792c2265c25897332c9, 贡永红)","(57a9e792c2265c25897332c9, 王德军)"
143,146,57a9e792c2265c25897332c9,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,南京鸿翥生物科技有限公司,段然,"(57a9e792c2265c25897332c9, 张新艳)","(57a9e792c2265c25897332c9, 南京鸿翥生物科技有限公司)","(张新艳, 段然)","(57a9e792c2265c25897332c9, 江苏省南京市中级人民法院)","(57a9e792c2265c25897332c9, 贡永红)","(57a9e792c2265c25897332c9, 段然)"
179,182,57a9e795c2265c25897332fd,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,王钲舜,王德军,"(57a9e795c2265c25897332fd, 张新艳)","(57a9e795c2265c25897332fd, 王钲舜)","(张新艳, 王德军)","(57a9e795c2265c25897332fd, 江苏省南京市中级人民法院)","(57a9e795c2265c25897332fd, 贡永红)","(57a9e795c2265c25897332fd, 王德军)"
180,183,57a9e795c2265c25897332fd,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,王钲舜,段然,"(57a9e795c2265c25897332fd, 张新艳)","(57a9e795c2265c25897332fd, 王钲舜)","(张新艳, 段然)","(57a9e795c2265c25897332fd, 江苏省南京市中级人民法院)","(57a9e795c2265c25897332fd, 贡永红)","(57a9e795c2265c25897332fd, 段然)"
181,184,57a9e795c2265c25897332fd,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,南京鸿翥生物科技有限公司,王德军,"(57a9e795c2265c25897332fd, 张新艳)","(57a9e795c2265c25897332fd, 南京鸿翥生物科技有限公司)","(张新艳, 王德军)","(57a9e795c2265c25897332fd, 江苏省南京市中级人民法院)","(57a9e795c2265c25897332fd, 贡永红)","(57a9e795c2265c25897332fd, 王德军)"
182,185,57a9e795c2265c25897332fd,2014-05-30,FALSE,江苏省南京市中级人民法院,二审,贡永红,NaN,0.0,江苏省,...,30,张新艳,南京鸿翥生物科技有限公司,段然,"(57a9e795c2265c25897332fd, 张新艳)","(57a9e795c2265c25897332fd, 南京鸿翥生物科技有限公司)","(张新艳, 段然)","(57a9e795c2265c25897332fd, 江苏省南京市中级人民法院)","(57a9e795c2265c25897332fd, 贡永红)","(57a9e795c2265c25897332fd, 段然)"
47382,47556,5ec834887116c36f1b9161b6,2018-08-21,TRUE,厦门市思明区人民法院,一审,(张希华),784.0,2572.0,福建省,...,21,孙亚,酉夕(厦门)餐饮管理有限公司,高晓宁,"(5ec834887116c36f1b9161b6, 孙亚)","(5ec834887116c36f1b9161b6, 酉夕(厦门)餐饮管理有限公司)","(孙亚, 高晓宁)","(5ec834887116c36f1b9161b6, 厦门市思明区人民法院)","(5ec834887116c36f1b9161b6, (张希华))","(5ec834887116c36f1b9161b6, 高晓宁)"
47383,47557,5ec834887116c36f1b9161b6,2018-08-21,TRUE,厦门市思明区人民法院,一审,(张希华),784.0,2572.0,福建省,...,21,孙亚,酉夕(厦门)餐饮管理有限公司,谢燕、商博雅,"(5ec834887116c36f1b9161b6, 孙亚)","(5ec834887116c36f1b9161b6, 酉夕(厦门)餐饮管理有限公司)","(孙亚, 谢燕、商博雅)","(5ec834887116c36f1b9161b6, 厦门市思明区人民法院)","(5ec834887116c36f1b9161b6, (张希华))","(5ec834887116c36f1b9161b6, 谢燕、商博雅)"


In [48]:
# fill graph data dict with selected cases
graph_data = { 'nodes': [], 'edges': [] }
for i in node_types:
    nodes = list(set(selected_cases[i]))
    for j in range(len(nodes)):
        if(type(nodes[j])==float): continue
        graph_data['nodes'].append({
            "id": str(i) + "_" + str(j),
            "label": str(i),
            "name": nodes[j], 
            "x": random.uniform(0,1),
            "y": random.uniform(0,1),
            "size": random.uniform(0.2,1)
        })
for i in edge_types:
    edges = list(set(selected_cases[i]))
    for j in range(len(edges)):
        source = edges[j][0]
        target = edges[j][1]
#         print(source, target, type(source), type(target))
        if(type(source)==float or type(target)==float): continue
        graph_data['edges'].append({
            "id": str(i) + "_" + str(j),
            "label": str(i),
            "source": source, 
            "target": target, 
        })

In [49]:
pd_nodes = pd.DataFrame(graph_data['nodes'])
pd_edges = pd.DataFrame(graph_data['edges'])

In [50]:
# pd_edges.to_dict(orient="record")

In [51]:
# assign source and target id in graph data edges
pd_edges['source_id'] = pd_edges['source'].apply(lambda r: pd_nodes[pd_nodes['name'] == r]['id'].values[0])
pd_edges['target_id'] = pd_edges['target'].apply(lambda r: pd_nodes[pd_nodes['name'] == r]['id'].values[0])
pd_edges.rename(columns={"source_id": "source", "target_id": "target"}, inplace=True)
pd_edges = pd_edges[['id', 'label', 'source', 'target']]

In [52]:
graph_data['edges'] = pd_edges.to_dict(orient="record")
pd.DataFrame(graph_data['edges'])

<ipython-input-52-1db4050a2e34>:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  graph_data['edges'] = pd_edges.to_dict(orient="record")
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


,id,label,source,target
0,case-plaintiff_0,case-plaintiff,case_0,plaintiff_1
1,case-plaintiff_1,case-plaintiff,case_2,plaintiff_1
2,case-plaintiff_2,case-plaintiff,case_1,plaintiff_0
3,case-defendant_0,case-defendant,case_2,defendant_0
4,case-defendant_1,case-defendant,case_0,defendant_2
5,case-defendant_2,case-defendant,case_1,defendant_1
6,case-defendant_3,case-defendant,case_2,defendant_2
7,case-defendant_4,case-defendant,case_0,defendant_0
8,plaintiff-lawyer_0,plaintiff-lawyer,plaintiff_1,lawyer_1
9,plaintiff-lawyer_1,plaintiff-lawyer,plaintiff_0,lawyer_3


In [53]:
# output graph data to json file
with open("graph_data.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(graph_data))

In [54]:
node_color_maps = {
    "case": "red", 
    "plaintiff": "black", 
    "defendant": "blue", 
    "lawyer": "yellow", 
    "court": "purple", 
    "judge": "green"
}
edge_color_maps = {
    "case-plaintiff": '#666', 
    "case-defendant": '#666', 
    "plaintiff-lawyer": '#666', 
    "case-court": '#666', 
    "case-judge": '#666', 
    "case-lawyer": '#666'
}

In [55]:
# with open("graph_data.json", "r") as f:
#     data = json.load(f)

<h2>4. Build graph with customed graph data</h2>

In [56]:
graph_data

{'nodes': [{'id': 'case_0',
   'label': 'case',
   'name': '57a9e795c2265c25897332fd',
   'x': 0.6442857818150665,
   'y': 0.9599013284481024,
   'size': 0.4979213761169728},
  {'id': 'case_1',
   'label': 'case',
   'name': '5ec834887116c36f1b9161b6',
   'x': 0.07053756418530299,
   'y': 0.4370331528431527,
   'size': 0.3583746642654325},
  {'id': 'case_2',
   'label': 'case',
   'name': '57a9e792c2265c25897332c9',
   'x': 0.7850935035916956,
   'y': 0.736931335226787,
   'size': 0.8525225411553061},
  {'id': 'plaintiff_0',
   'label': 'plaintiff',
   'name': '孙亚',
   'x': 0.35740345727998113,
   'y': 0.30701792417728735,
   'size': 0.2942997594018891},
  {'id': 'plaintiff_1',
   'label': 'plaintiff',
   'name': '张新艳',
   'x': 0.8608294170418769,
   'y': 0.13125471830189017,
   'size': 0.4366265281104272},
  {'id': 'defendant_0',
   'label': 'defendant',
   'name': '王钲舜',
   'x': 0.018571662699930602,
   'y': 0.39242939916784403,
   'size': 0.2106990040830711},
  {'id': 'defendant_1',

In [57]:
html_template = Template('''
<style>
  #graph-container {
      height: 800px;
  };
</style>
<div id="graph-container"></div>
<div id="message1"></div>
<div id="message2"></div>
<script>$js_text</script>
''') 

In [62]:
# 重新分配节点的位置，让其更美观
js_text_tepmlate = Template('''
var node_color_maps = {
    "case": '#D98880', 
    "plaintiff": '#9B59B6', 
    "defendant": '#85C1E9', 
    "lawyer": '#27AE60', 
    "court": '#F1C40F', 
    "judge": '#5D6D7E'
};

var edge_color_maps = {
    "case-plaintiff": '#D98880', 
    "case-defendant": '#D98880', 
    "plaintiff-lawyer": '9B59B6', 
    "case-court": '#D98880', 
    "case-judge": '#D98880', 
    "case-lawyer": '#D98880'
};

var g = $graph_data;

s = new sigma({
  graph: g,
  renderer: {
    container: document.getElementById('graph-container'),
    type: 'canvas'
  },
  settings: {
    minNodeSize: 1,
    maxNodeSize: 10,
    minEdgeSize: 0.1,
    maxEdgeSize: 2,
    enableEdgeHovering: true,
    edgeHoverSizeRatio: 2, 
  }
});

if (!(s.graph['neighbors'])){
    sigma.classes.graph.addMethod('neighbors', function(nodeId) {
    var k,
        neighbors = {},
        index = this.allNeighborsIndex[nodeId] || {};

    for (k in index)
      neighbors[k] = this.nodesIndex[k];

    return neighbors;
    })
}
else{
    console.log("neighbors function not in s.graph!")
};

s.graph.nodes().forEach(function(n){
    n.originalColor = node_color_maps[n.label]
});

s.graph.edges().forEach(function(e){

    e.originalColor = edge_color_maps[e.label]
});

s.bind('clickNode', function(e){
    var nodeId = e.data.node.id, 
        toKeep = s.graph.neighbors(nodeId);
    toKeep[nodeId] = e.data.node;
    s.graph.nodes().forEach(function(n){
        if(toKeep[n.id])
            n.color = node_color_maps[n.label];
        else
            n.color = '#eee';
    });
    s.graph.edges().forEach(function(n){
        if(toKeep[n.source] && toKeep[n.target])
            n.color = node_color_maps[n.label];
        else
            n.color = '#eee';
    });
    s.refresh();
});

s.bind('clickStage', function(e){
    s.graph.nodes().forEach(function(n){
        n.color = node_color_maps[n.label]
    });
    s.graph.edges().forEach(function(n){
        n.color = edge_color_maps[n.label]
    });
    s.refresh();
});

s.bind('overNode', function(e){
    var message1 = e.data.node.id.toString();
    var message2 = e.data.node.name.toString();
    document.getElementById('message1').innerHTML = message1 + " " + message2;
});
''')

In [63]:
js_text = js_text_tepmlate.substitute({
    "graph_data": json.dumps(graph_data)
})

In [64]:
html_text = html_template.substitute({
    "js_text": js_text
})

In [65]:
HTML(html_text)